# 0 Preface
このノートブックではニューラルネットワークを用いて，以下の3つの基本的な問題に取り組みます．


*   二値分類
*   多クラス分類
*   回帰

ノートブック中のコードは (https://github.com/fchollet/deep-learning-with-python-notebooks) のコードを一部改変して使用しました．


初期設定のnumpyのバージョンでは，後ほど行うデータセットのダウンロードがうまくいかないため，ここではnumpyのバージョンを1.16.2とします．
<br>
エラーが出ますが，このハンズオンの実行には影響ありません．
<br>
(参考) https://github.com/tensorflow/tensorflow/issues/28102

なお，この設定は現在実行中のcolabのインスタンスのみに適用されます．
<br>
(colabのインスタンスは生成されてから12時間もしくはセッションが切れてから90分でリセットされます．)

In [0]:
!pip install numpy==1.16.2
# !pip install numpy==1.16.2 folium==0.2.1 imgaug==0.2.5

Warningに書かれているように，ランタイムを再起動する必要があります．
[RESTART RUNTIME] をクリックしてください．

In [0]:
import tensorflow as tf
import keras

In [0]:
print(tf.test.gpu_device_name())

'/device:GPU:0'と表示されればGPUが使えています．

# 1 二値分類の例：映画レビューの分類

この章では，映画レビューのテキストの内容に基づいて，映画レビューを肯定的なレビューと否定的なレビューに分類します．

## 1.1 IMDb データセット

IMDb データセットはIMDb (Internet Movie Database) から収集された，50,000件の映画のレビューで構成されています．各レビューには，「肯定的」もしくは「否定的」のラベルが付けられており，それぞれ半々ずつあります．また，このデータセットは訓練用の25,000件のレビューとテスト用の25,000件のレビューに分かれています．

MNISTデータセットと同様に，IMDbデータセットも前処理された状態でKerasに含まれています．レビューの内容（単語のシーケンス）は整数のシーケンスに変換されており，各整数はそれぞれ辞書の特定の単語を指しています．

In [0]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

* `train_data` : 訓練データのレビューのリスト
* `test_data` : テストデータのレビューのリスト
* `train_labels` : 訓練データのラベルのリスト
* `test_labels` : テストデータのラベルのリスト

まずは，データセットの中身を確認してみましょう．

In [0]:
train_data[0]

各レビューは単語のインデックス（単語をエンコードしたもの）からなるリストです．

In [0]:
train_labels[0]

ラベルは0, 1の二値です．0は「否定的」，1は「肯定的」を意味します．

ここでは，訓練データにおいて出現頻度が高い10,000個の単語のみを用い，出現頻度が低い単語は捨てています．そのため，単語のインデックスは10,000未満となっています．

In [0]:
max([max(sequence) for sequence in train_data])

参考までに，レビューをデコードして内容を確認してみましょう．

In [0]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [0]:
decoded_review

## 1.2 データの前処理

整数のリストをそのままニューラルネットワークに入力することはできません．ここでは，整数のリストをone-hotエンコーディングによって，10,000次元の2値ベクトルに変換します．
例えば，[3,5]というリストだった場合，3番目と5番目の要素が1でそれ以外は全て0の10,000次元ベクトルに変換します．

In [0]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

レビューは次のような2値のnumpy配列に変換されています．

In [0]:
x_train[0]

ラベルも同様にnumpy配列に変換します．

In [0]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## 1.3 ネットワークの構築

ここでは下の図のような，全結合(Dense)層3層からなるネットワークを構築します．
なお，使用する層の数，各層の出力ユニット数については，改善の余地があります．

![3-layer network](https://s3.amazonaws.com/book.keras.io/img/ch3/3_layer_network.png)

Denseのコンストラクタの引数はその層の出力ユニットの数に対応し，キーワード引数は活性化関数と入力テンソルの形状に対応します．
入力テンソルの形状は，バッチ次元を含まないことに注意してください．

中間層の出力ユニット数は16とし，活性化関数としてReLUを用います．
最終層の出力ユニット数は1とし，活性化関数としてシグモイド関数を用います．
シグモイド関数の出力は 0~1のスカラー値であり，そのレビューが肯定的である確率として解釈できます．

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

最後にモデルのコンパイルを行います．
二値分類問題では，損失関数として`binary_crossentropy`を用いるのが一般的です．
また，オプティマイザとして`rmsprop`を用います．さらに，訓練時に正解率も監視します．

In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

## 1.4 アプローチの検証

モデルの訓練時にテストデータを用いて評価を行うことは適切ではありません．
<br>
ここでは，訓練時に，訓練データに含まれていない新しいデータにおける正解率を監視するため，元の訓練データセットから取り分けておいた10,000個のサンプルを使って検証データセットを作成します．

In [0]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

ミニバッチサイズは512とし，20エポックの訓練を行います．同時に，検証データにおける損失値と正解率を監視します．なお，検証データは引数`validation_data`で指定します．

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

`model.fit()`は`History`オブジェクトを返します．このオブジェクトには`history`というメンバーがあり，訓練中に観測された全てのデータがディクショナリとして保存されています．

In [0]:
history_dict = history.history
history_dict.keys()

このディクショナリには，各エポックでの訓練データ，検証データそれぞれにおける正解率，損失値が含まれています．
matplotlibを用いてプロットしてみましょう．

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

点は訓練データでの結果，折れ線は検証データでの結果を表しています．
各図から，訓練データでの損失値はエポックごとに小さくなり，正解率はエポックごとに向上していることが分かります．
しかし，検証データでの損失値と正解率はそうはなっていません．

これは過学習に陥っているためです．結局のところ，このモデルは訓練データに特化した表現を学習しているだけであり，訓練データ以外のデータに対して汎化していません．

こういった場合，訓練を早期に中止することで，過学習を抑制することができます．
新しいモデルを4エポックで訓練し，テストデータで評価してみましょう．

In [0]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [0]:
results

このかなり単純なアプローチでも，88%ほどの正解率が達成されています．最先端のアプローチでは，95%近い正解率を達成することが可能です．

## 1.5 新しいデータでの予測

訓練済みのモデルを用いてテストデータでのレビューの評価を予測してみましょう．`predict`メソッドを用いることでレビューが肯定的だと判断される確率を計算できます．

In [0]:
model.predict(x_test)

このモデルでは，確実に分類できるサンプル (0.99以上や0.01以下) もあれば，そうではないサンプルもあるようです．

## 1.6 過学習の対策

### 1.6.1 重みの正則化
ネットワークの重みに小さい値だけが設定されるようにすることで，ネットワークが複雑になりすぎないようになり過学習を防ぐことができる可能性があります．

重みの値をそのように制限することを **重みの正則化** と言います．重みを正則化するためには，重みの値に対するコストをネットワークの損失関数に加えます．代表的なコストには次のようなものがあります．
- L1正則化

  追加されるコストは重み係数の絶対値(重みのL1ノルム)に比例します．
- L2正則化

  追加されるコストは重み係数の値の二乗(重みのL2ノルム)に比例します．
  
先ほどまで作成していたネットワークにL2正則化を追加してみましょう．

In [0]:
from keras import regularizers

x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

L2正則化を加えていない時に比べて，訓練データでのlossが大きい代わりに検証データでのロスが小さくなっていることが確認できると思います．

### 1.6.2 ドロップアウト

**ドロップアウト**とはニューラルネットワークにおいて最もよく使われている正則化手法の一つです．ドロップアウトを層に適用すると，訓練中にその層の出力特徴量の一部がランダムにdropされ0になります．
例えば，ある層の出力が[0.2, 0.5, 0.8, 1.3, 0.3]だったとして，ドロップアウトをその層に適用すると出力は[0, 0.5, 0, 1.3, 0.3]のようになります．(この結果はあくまで一例であり，0になる要素はランダムに選ばれます．)

**ドロップアウト率**とは0にする要素の割合であり，通常0.2から0.5に設定されます．テスト時には，ドロップアウトは適用されません．そのため，訓練時よりも多くのユニットが活性化されてしまいます．これに折り合いをつけるため，テスト時にはドロップアウト率と同じ割合でスケールダウンされます．例えば，ドロップアウト率0.2で学習したならテスト時の出力は0.8倍されます．しかし，kerasなどのフレームワークを用いる際はライブラリが勝手にスケールダウンしてくれるため意識する必要はありません．映画分類ネットワークにドロップアウトを二つ追加してみましょう．

In [0]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

こちらも，ドロップアウトを適用していない時に比べて訓練データでのlossが大きく検証データでのlossが小さくなっていることが確認できると思います．

# 2 多クラス分類の例：ニュース配信の分類

前章では2クラスの場合の分類問題を扱いましたが，ここではクラスが3つ以上ある場合を考えます．
この章では，Reutersのニュース配信を46種類のトピックに分類します．

## 2.1 Reuters データセット


Reuters データセットは，1986年にReutersによって配信された短いニュース記事とそれらのトピックを集めたものです．トピックは46種類あります．トピック間でのサンプル数に偏りがあるものの，各トピックのサンプルが少なくとも10個は含まれています．
IMDbデータセットやMNISTデータセットと同様に，Reutersデータセットも前処理された状態でKerasに含まれています．

In [0]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

先ほどのIMDbデータセットの例と同様に，訓練データにおいて出現頻度が高い10,000個の単語のみを用い，出現頻度が低い単語は捨てています．
訓練サンプルは8,982個，テストサンプルは2,246個になります．

In [0]:
len(train_data)

In [0]:
len(test_data)

IMDbデータセットのレビューと同様に，各サンプルは単語のインデックスのリストです．

In [0]:
train_data[10]

参考までに，レビューをデコードして内容を確認してみましょう．

In [0]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [0]:
decoded_newswire

サンプルに関連づけられているラベルは，0〜45の整数 (トピックインデックス) です．

In [0]:
train_labels[10]

## 2.2 データの準備

データのベクトル化は，IMDbデータセットの場合と同様の方法で行います．

In [0]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

ラベルは，one-hotエンコーディングによりベクトルします．

In [0]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Our vectorized training labels
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(test_labels)

なおこの処理はKerasに組み込まれている関数`to_categorical`でも実行可能です．

In [0]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

## 2.3 ネットワークの構築

多クラス分類の場合，ネットワークの出力が各クラスに属する確率に対応するため，最終層のユニット数はクラス数となります．
ここでは，46種類のクラスを学習するため，前章の例より出力空間の次元がはるかに大きくなります．
そのため，前章の例のように中間層を16次元とした場合，中間層が情報ボトルネックとなり，重要な情報が失われてしまう恐れがあります．
したがって，ここではもっと大きな64ユニットの中間層を用います．

また，最終層の活性関数は`Softmax`とします．`i`番目の出力がクラス`i`に属している確率を表し，また確率値の合計が`1`となります．

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

多クラス分類では，損失関数として`categorical_crossentropy`を用いるのが一般的です．

In [0]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 2.4 アプローチの検証

訓練データのうち1,000サンプルを検証データセットとして使用するために分けておきます．

In [0]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

次に，512サンプルのミニバッチで20エポックの訓練を行います．

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

最後に，損失値と正解率をプロットします．

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure

acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

8エポックの後，このネットワークは過学習に陥っています．新しいネットワークを8エポックで訓練し，その後テストデータで評価してみましょう．

In [0]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=8,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

In [0]:
results

このアプローチの正解率は78%程度となりました．

In [0]:
import copy

test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
float(np.sum(np.array(test_labels) == np.array(test_labels_copy))) / len(test_labels)

ランダムなベースラインの正解率は19%程度となっています．そのため78%という結果は，よい結果であるといえます．

## 2.5 新しいデータでの予測

このモデルの`predict`メソッドは，46種類ある各トピックの確率分布を返します．このことを検証するために，テストデータ全体のトピック予測を計算してみましょう．

In [0]:
predictions = model.predict(x_test)

`predictions`の各エントリは，長さが46のベクトルです．

In [0]:
predictions[0].shape

このベクトルの和は1になります．

In [0]:
np.sum(predictions[0])

最大となるエントリが，確率が最も高いクラス，つまり予測クラスです．

In [0]:
np.argmax(predictions[0])

## 2.5 ラベルと損失値を処理する別の方法

ラベルをone-hotエンコードするのではなく，整数のテンソルとしてキャストすることも可能です．

In [0]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

この場合，損失関数として，one-hotエンコーディングの際に用いた`categorical_crossentropy`ではなく，`sparse_categorical_crossentropy`を用います．

In [0]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

この損失関数は，数学的には`sparse_categorical_crossentropy`と同じで，違いはインターフェイスの部分だけです．

## 2.6 十分な大きさの中間層を持つことの重要性

最終的な出力は46次元であるため，ユニットの数が46よりもずっと小さい中間層は避けるべきです．
たとえば4次元の中間層を用いた場合はどうなるでしょうか．実際に試してみましょう．

In [0]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))

検証データセットでの正解率のピークは71%程度であり，先ほどよりも低い値となっています．これは，46クラスの分離超平面を復元するのに必要な情報を，あまりにも小さな中間層に詰め込もうとしたことが原因です．このネットワークには，これらの表現に必要な情報の大部分は詰め込むことができますが，必要な情報がすべて詰め込まれるわけではありません．

# 3 回帰の例：住宅価格の予測

これまでの章では，離散的なラベルを予測する分類問題を取り上げました．この章では，連続値を予測する回帰問題を扱います．

## 3.1 Boston Housing データセット

ここでは，1970年代中頃のボストン近郊での住宅価格の中央値を予測します．この予測には，犯罪発生率や地方財産税の税率など，当時のボストン近郊に関するデータを使用します．

このデータセットに含まれているデータは506個と比較的少なく，404の訓練サンプルと102のテストサンプルに分割されています．また，入力データの特徴量はそれぞれ異なる尺度を用いています．たとえば，割合を0〜1の値で表すものもあれば，1〜12の値をとるものや，0〜100の値をとるものもあります．

In [0]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [0]:
train_data.shape

In [0]:
test_data.shape

各サンプルは13種類の数値の特徴量で構成されています．これらの特徴量は，犯罪発生率，1戸あたりの平均部屋数，幹線道路へのアクセス指数などを表します．

In [0]:
train_targets

目標値は，住宅価格の中央値 (1,000ドル単位) です．住宅価格は主に10,000ドルから50,000ドルの間です．

## 3.2 データの正規化

異なる範囲の値をとる特徴量を用いて学習を行うことは困難です．そこで，各特徴量ごとに正規化を行います．

具体的には各特徴量ごとに，平均値を引き，標準偏差で割ることによって各特徴量の平均を0，標準偏差を1に揃えます．

In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## 3.3 ネットワークの構築

訓練データが少ないため，2つの中間層をもつ小さなニューラルネットワークを用います．中間層のユニット数は64とします．一般的に，訓練データが少なければ少ないほど，モデルは過学習に陥りやすくなります．小さいネットワークを使用することで，過学習を抑制することができます．

In [0]:
from keras import models
from keras import layers

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

最終層のユニットは1つだけで，活性化関数を適用していません．この場合，最終層は完全に線形の層となり任意の値を予測することが可能です．

また，損失関数として平均二乗誤差 (mean square error, MSE) を用いています．これは，予測値と目的値との差の自乗であり，回帰問題の損失関数として広く使用されています．

訓練時には，平均絶対誤差 (mean absolute error, MAE) という新しい指標も監視しています．これは，予測値と目的値との差の絶対値です．例えば，MAEが0.5のとき，予測値は平均で500ドルずれていることになります．

## 3.4 k分割交差検証によるアプローチの検証

これまでの章では，データを訓練データセットと検証データセットに分割することでネットワークの評価を行ってきました．
しかし今回の場合のように，データの数が少ない場合，検証データセットも小さくなり，検証データセットの分割方法によっては，検証スコアのバリアンスが高くなり，過学習に陥ります．これでは，モデルを正確に評価することは不可能です．

こうした状況では，k分割交差検証が適しています．k分割交差検証では，利用可能なデータをK個のフォールドに分割し，まったく同じモデルのインスタンスをK個作成します．そして，各モデルをK-1個のフォールドで訓練し，残りの1個のフォールドで評価します．そして最後に，K個の検証スコアの平均を求めます．通常，Kの値は4か5とします．

In [0]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=16, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

In [0]:
all_scores

In [0]:
np.mean(all_scores)

フォールドによって検証スコアにばらつきがあることがわかります．k分割交差検証ではそれらの平均値を検証スコアとします．この場合，誤差の平均は2,400ドルです．住宅価格が10,000ドルから50,000ドルであることを考えると，これはかなり大きな誤差です．

このネットワークをもう500エポック訓練してみましょう．

In [0]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [0]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

続いて，すべてのフォールドを対象に，エポックごとのMAEスコアの平均を求めます．

In [0]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

結果をプロットしましょう．

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

スケーリングの問題やバリアンスが比較的高いせいで，このプロットを読んで理解するのは少々難しいですね．そこで，次のようにしてみましょう．
* 最初の10個のデータ点を省略する (尺度が異なるため)
* 各データ点をその手前にあるデータ点の指数移動平均に置き換えることで，なめらかな曲線にする．

In [0]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

このプロットから，検証スコア (MAE) が80エポック程度のときに最適になっていることがわかります．その後は過学習に陥っています．

モデルの他のパラメータのチューニングが完了したら，最適なパラメータと訓練データ全体を使って最終的なモデルの訓練をします．続いて，テストデータでの性能を調べます．

In [0]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [0]:
test_mae_score

依然として2,500ドル程度もずれています．